# About the Data

* The data was taken from 
the csv file complaints.csv


* The complaints are for the products:<br>

  1. 'Bank account or service'
  2. 'Checking or savings account'
  3. 'Consumer Loan'
  4. 'Credit card or prepaid card'
  5. 'Credit reporting, credit repair services, or other personal consumer reports' 
  6. 'Debt collection'
  7. Money transfer/s, virtual currency, or money service'
  8. 'Mortgage'
  9. 'Payday loan, title loan, or personal loan'
  10. 'Student loan'
  11. 'Vehicle loan or lease'


* The data cleaning was done using pandas


## Next Steps

* Use Spacy library to preprocess data.

* Finetune DistilBERT on the sample data 

## Google Drive access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# write the appropriate paths to retrieve the data and store results 
data_path = '/content/drive/MyDrive/complaints.csv'

# Loading the dataset

In [3]:
#Load the data
import pandas as pd

df1 = pd.read_csv(data_path, dtype={"Consumer complaint narrative": "string", "Consumer consent provided?": "string", "Timely response?":"string"})
df1 = df1[['Consumer complaint narrative','Product', 'State']]
df1 = df1[df1['State'] != 'ND']
df1.rename(columns = {'Product':'product', 'Consumer complaint narrative':'consumer_complaint_narrative'}, inplace = True)
df1 = df1[df1.consumer_complaint_narrative.notnull()]
df2 = df1.copy()#changed
df2.reset_index(inplace = True, drop = True)
#reduce categories
df2.loc[df2['product'] == 'Credit reporting', 'product'] = 'Credit reporting, credit repair services, or other personal consumer reports'
df2.loc[df2['product'] == 'Credit card', 'product'] = 'Credit card or prepaid card'
df2.loc[df2['product'] == 'Prepaid card', 'product'] = 'Credit card or prepaid card'# new category

df2.loc[df2['product'] == 'Payday loan', 'product'] = 'Payday loan, title loan, or personal loan'
df2.loc[df2['product'] == 'Virtual currency', 'product'] = 'Money transfer/s, virtual currency, or money service'
df2.loc[df2['product'] == 'Money transfers', 'product'] = 'Money transfer/s, virtual currency, or money service'# new category
df2.loc[df2['product'] == 'Money transfer, virtual currency, or money service', 'product'] = 'Money transfer/s, virtual currency, or money service'
df2 = df2[df2['product'] != 'Other financial service']

df2.reset_index(inplace = True, drop = True)
df2.head()


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,consumer_complaint_narrative,product,State
0,transworld systems inc. is trying to collect ...,Debt collection,FL
1,"Over the past 2 weeks, I have been receiving e...",Debt collection,NC
2,Pioneer has committed several federal violatio...,Debt collection,CA
3,I started the process to refinance my current ...,Mortgage,NC
4,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...","Credit reporting, credit repair services, or o...",CA


In [4]:
#Print the products counts
df2['product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    265093
Debt collection                                                                 130316
Mortgage                                                                         72886
Credit card or prepaid card                                                      67978
Checking or savings account                                                      27705
Student loan                                                                     27390
Bank account or service                                                          14867
Money transfer/s, virtual currency, or money service                             13714
Vehicle loan or lease                                                            11651
Payday loan, title loan, or personal loan                                        10365
Consumer Loan                                                                     9463
Name: product, dtype: int64

In [5]:
#Split 'consumer_complaint_narrative' into substrings whenever whitespace occur
df2['split_words_whitespaces'] = df2['consumer_complaint_narrative'].apply(lambda x: x.split())
#Count the number of substrings in 'split_words_whitespaces'
df2['number_of_words'] = df2['split_words_whitespaces'].apply(lambda x: len(x))
#Count the number of charachters in  'consumer_complaint_narrative'
df2['number_of_charachters'] = df2['consumer_complaint_narrative'].apply(lambda x: len(x))
#Calculate the ratio of number of charachters by number of words
df2['charachters_by_words'] = df2['number_of_charachters'] // df2['number_of_words']
#Count the number of unique strings in 'split_words_whitespaces'
df2['number_of_unique_words'] = df2['split_words_whitespaces'].apply(lambda x : len(set(x)))
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651428 entries, 0 to 651427
Data columns (total 8 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   consumer_complaint_narrative  651428 non-null  string
 1   product                       651428 non-null  object
 2   State                         647916 non-null  object
 3   split_words_whitespaces       651428 non-null  object
 4   number_of_words               651428 non-null  int64 
 5   number_of_charachters         651428 non-null  int64 
 6   charachters_by_words          651428 non-null  int64 
 7   number_of_unique_words        651428 non-null  int64 
dtypes: int64(4), object(3), string(1)
memory usage: 39.8+ MB


In [6]:
#Retain certain records only
df3 = df2.query('(number_of_charachters > 60)&(number_of_unique_words > 30)').drop(columns=['State'])
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574800 entries, 1 to 651427
Data columns (total 7 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   consumer_complaint_narrative  574800 non-null  string
 1   product                       574800 non-null  object
 2   split_words_whitespaces       574800 non-null  object
 3   number_of_words               574800 non-null  int64 
 4   number_of_charachters         574800 non-null  int64 
 5   charachters_by_words          574800 non-null  int64 
 6   number_of_unique_words        574800 non-null  int64 
dtypes: int64(4), object(2), string(1)
memory usage: 35.1+ MB


In [7]:
#Define the strings to mask
mask_words_list =['XX/XX/XXXX','XX-XX-XXXX', #DATE mm/dd/yyyy mm-dd-yyyy
                  'XXXX XXXX XXXX XXXX XXXX','XXXX-XXXX-XXXX-XXXX',#CREDIT or PREPASID CARD NUMBER
                  'XXXX XXXX XXXX XXXX','XXXX XXXX XXXX','XXXX-XXXX-XXXX','XXXX-XXXX','XXXX XXXX',
                  'XXX-XX-XXXX','XXX-XXX','XX-XXXX',
                  'XXXXXXXXXXXXXXXXXX','XXXXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXX',# BANK ACCOUNT NUMBER
                  'XXXXXXXXXXXXX', 'XXXXXXXXXXXX', 'XXXXXXXXXXX',                                                   # RANGES FROM 12 TO 18 DIGITS
                  'XXXXXXXXXX','XXXXXXXXX'          #ROUTING NUMBER IS 9 DIGIT
                  'XXXX','XXX','XX']

In [8]:
#Get Potential mask words
def get_potential_maskwords(example,mask_words_list):
  mask_list = [string for string in example if any(substring in string for substring in mask_words_list)]
  return mask_list

from tqdm import tqdm, tqdm_notebook

# instantiate
tqdm.pandas()

df3['potenial_mask_words']= df3['split_words_whitespaces'].progress_apply(lambda x : get_potential_maskwords(x,mask_words_list))
print("\n\nGetting Potential Mask Words Completed")

100%|██████████| 574800/574800 [04:14<00:00, 2258.37it/s]




Getting Potential Mask Words Completed


In [9]:
#Calculate number of potential mask words
df3['number_of_potenial_mask_words'] = df3['potenial_mask_words'].apply(lambda x: len(x))
#Calculate the ratio of number of potential mask words by total words
df3['potenial_mask_words_BY_words'] = df3['number_of_potenial_mask_words']/df3['number_of_words']

In [10]:
#Retain certain records only
df4 = df3.query('potenial_mask_words_BY_words < = 0.5')
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 573302 entries, 1 to 651427
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   consumer_complaint_narrative   573302 non-null  string 
 1   product                        573302 non-null  object 
 2   split_words_whitespaces        573302 non-null  object 
 3   number_of_words                573302 non-null  int64  
 4   number_of_charachters          573302 non-null  int64  
 5   charachters_by_words           573302 non-null  int64  
 6   number_of_unique_words         573302 non-null  int64  
 7   potenial_mask_words            573302 non-null  object 
 8   number_of_potenial_mask_words  573302 non-null  int64  
 9   potenial_mask_words_BY_words   573302 non-null  float64
dtypes: float64(1), int64(5), object(3), string(1)
memory usage: 48.1+ MB


In [11]:
#Calculate the product frequency ratio to sample data accordingly
df4_product_freqdict = (df4['product'].value_counts().sort_index()/df4['product'].count()).to_dict()

In [12]:
#Define the word threshold and retain data within the threshold
word_threshold = 500
df4_within_threshold= df4[ df4['number_of_words'].le(word_threshold)].sort_values('product')
df4_within_threshold.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 528494 entries, 525589 to 390410
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   consumer_complaint_narrative   528494 non-null  string 
 1   product                        528494 non-null  object 
 2   split_words_whitespaces        528494 non-null  object 
 3   number_of_words                528494 non-null  int64  
 4   number_of_charachters          528494 non-null  int64  
 5   charachters_by_words           528494 non-null  int64  
 6   number_of_unique_words         528494 non-null  int64  
 7   potenial_mask_words            528494 non-null  object 
 8   number_of_potenial_mask_words  528494 non-null  int64  
 9   potenial_mask_words_BY_words   528494 non-null  float64
dtypes: float64(1), int64(5), object(3), string(1)
memory usage: 44.4+ MB


In [13]:
#Get Sample of the data frame
df4_within_threshold_group=df4_within_threshold.groupby('product', group_keys=False)
sample_size= 55000# TRY CHANGING TO 750000
df4_within_threshold_sample = pd.concat(dff.sample(n=int(df4_product_freqdict.get(i)* sample_size)) for i,dff in df4_within_threshold_group)
df4_within_threshold_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54994 entries, 448280 to 159822
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   54994 non-null  string 
 1   product                        54994 non-null  object 
 2   split_words_whitespaces        54994 non-null  object 
 3   number_of_words                54994 non-null  int64  
 4   number_of_charachters          54994 non-null  int64  
 5   charachters_by_words           54994 non-null  int64  
 6   number_of_unique_words         54994 non-null  int64  
 7   potenial_mask_words            54994 non-null  object 
 8   number_of_potenial_mask_words  54994 non-null  int64  
 9   potenial_mask_words_BY_words   54994 non-null  float64
dtypes: float64(1), int64(5), object(3), string(1)
memory usage: 4.6+ MB


In [17]:
#Reset the indices in the df4_within_threshold_sample data
sample_df = df4_within_threshold_sample.reset_index(drop=True) 
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54994 entries, 0 to 54993
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   consumer_complaint_narrative   54994 non-null  string 
 1   product                        54994 non-null  object 
 2   split_words_whitespaces        54994 non-null  object 
 3   number_of_words                54994 non-null  int64  
 4   number_of_charachters          54994 non-null  int64  
 5   charachters_by_words           54994 non-null  int64  
 6   number_of_unique_words         54994 non-null  int64  
 7   potenial_mask_words            54994 non-null  object 
 8   number_of_potenial_mask_words  54994 non-null  int64  
 9   potenial_mask_words_BY_words   54994 non-null  float64
dtypes: float64(1), int64(5), object(3), string(1)
memory usage: 4.2+ MB


## Download the Data

In [18]:
#Download the sample data
sample_df.to_csv("SAMPLE_21_APRIL_2022.csv", encoding='utf-8', index=False)